In [1]:
import math

import pandas as pd
import mwclient

from lol_fandom import SITE
from lol_fandom import get_leagues, get_tournaments
from lol_fandom import get_scoreboard_games, get_scoreboard_players
from lol_fandom import from_response

pd.set_option('display.max_columns', None)

In [2]:
class Team:
    q = math.log(10) / 400
    def __init__(self, name, league):
        self.name = name
        self.league = league
        self.win = 0
        self.loss = 0
        self.r = 1000
        self.RD = 350

    def get_g(RDi):
        return 1 / math.sqrt(1 + (3 * Team.q ** 2 * RDi ** 2) / math.pi ** 2)

    def get_e(r0, ri, g):
        return 1 / (1 + 10 ** ((g * (r0 - ri)) / -400))

    def get_d(g, e):
        return 1 / (Team.q ** 2 * g ** 2 * e * (1 - e))
    
    def update_point(team1, team2, result):
        # team1 win - result = 1 team1 loss - result = 0
        assert isinstance(team1, Team)
        assert isinstance(team2, Team)

        team1_r = team1.r
        team2_r = team2.r
        team1_RD = team1.RD
        team2_RD = team2.RD

        team1._update_point(team2_r, team2_RD, result)
        team2._update_point(team1_r, team1_RD, 1 - result)

    def _update_point(self, ri, RDi, s):
        if s == 1:
            self.win += 1
        else:
            self.loss += 1

        g_RD = Team.get_g(RDi)
        e = Team.get_e(self.r, ri, g_RD)
        d_2 = Team.get_d(g_RD, e)
        self.r = self.r + Team.q / (1 / self.RD ** 2 + 1 / d_2) * g_RD * (s - e)

        self.RD = math.sqrt((1 / self.RD ** 2 + 1 / d_2) ** -1)

    def get_win_prob(self, opponent):
        return Team.get_e(self.r, opponent.r, Team.get_g(opponent.RD))

    def to_dict(self):
        data = {
            'League': self.league,
            'Win': self.win,
            'Loss': self.loss,
            'WinRate': self.win / (self.win + self.loss) if self.win != 0 else 0,
            'r': self.r,
            'RD': self.RD
        }

        return data


In [3]:
def proceed_rating(teams, games):
    for row in games.itertuples():
        team1, team2 = row.Team1, row.Team2
        result = 1 if row.WinTeam == team1 else 0
        Team.update_point(teams[team1], teams[team2], result)

def get_rating(teams):
    ratings = pd.DataFrame(
        data=map(lambda x: x.to_dict(), teams.values()),
        index=teams.keys()
    )
    ratings = ratings.sort_values(by='r', ascending=False)
    return ratings

In [4]:
leagues = get_leagues()
leagues

,League,League Short,Region,Level,IsOfficial
0,2015 All-Star Event,2015 ASE,International,Showmatch,Yes
1,2015 International Wildcard Tournament,2015 IWCT,International,Primary,Yes
2,2016 International Wildcard Qualifier,IWCQ,International,Primary,Yes
3,All-Star,All-Star,International,Showmatch,Yes
4,Arena of Legends,AOL,North America,Secondary,No
...,...,...,...,...,...
120,Vietnam Championship Series,VCS,Vietnam,Primary,Yes
121,Volcano League,VL,Latin America,Secondary,Yes
122,World Championship,WCS,International,Primary,Yes
123,World Cyber Arena,WCA,China,Primary,No


In [5]:
target_leagues = ['LCK', 'LPL', 'LEC', 'LCS']

In [6]:
tournaments = pd.DataFrame()
for league in target_leagues:
    t = get_tournaments(f'L.League_Short="{league}" and T.Year=2022')
    tournaments = pd.concat([tournaments, t])
tournaments = tournaments.sort_values(by='DateStart').reset_index(drop=True)
tournaments

,Name,OverviewPage,DateStart,Date,League,Region,EventType,StandardName,Split,SplitNumber,TournamentLevel,IsQualifier,IsPlayoffs,IsOfficial,Year,DateStart__precision,Date__precision
0,LPL 2022 Spring,LPL/2022 Season/Spring Season,2022-01-10,2022-03-25,Tencent LoL Pro League,China,Offline/Online,LPL 2022 Spring,Spring,1,Primary,0,0,1,2022,1,1
1,LCK 2022 Spring,LCK/2022 Season/Spring Season,2022-01-12,2022-03-20,LoL Champions Korea,Korea,Offline,LCK 2022 Spring,Spring,1,Primary,0,0,1,2022,1,1
2,LCS 2022 Lock In,LCS/2022 Season/Lock In,2022-01-14,2022-01-30,League of Legends Championship Series,North America,None,LCS 2022 Lock In,Spring,1,Primary,0,1,1,2022,1,1
3,LEC 2022 Spring,LEC/2022 Season/Spring Season,2022-01-14,2022-03-06,LoL European Championship,Europe,Online,LEC 2022 Spring,Spring,1,Primary,0,0,1,2022,1,1
4,LCS 2022 Spring,LCS/2022 Season/Spring Season,2022-02-05,2022-03-27,League of Legends Championship Series,North America,None,LCS 2022 Spring,Spring,1,Primary,0,0,1,2022,1,1
5,LCK 2022 Spring Playoffs,LCK/2022 Season/Spring Playoffs,2022-03-23,2022-04-02,LoL Champions Korea,Korea,Online/Offline,LCK 2022 Spring Playoffs,Spring,1,Primary,0,1,1,2022,1,1
6,LEC 2022 Spring Playoffs,LEC/2022 Season/Spring Playoffs,2022-03-25,2022-04-10,LoL European Championship,Europe,None,LEC 2022 Spring Playoffs,Spring,1,Primary,0,1,1,2022,1,1
7,LPL 2022 Spring Playoffs,LPL/2022 Season/Spring Playoffs,2022-03-26,2022-04-23,Tencent LoL Pro League,China,Online,LPL 2022 Spring Playoffs,Spring,1,Primary,0,1,1,2022,1,1
8,LCS 2022 Spring Playoffs,LCS/2022 Season/Spring Playoffs,2022-04-02,2022-04-24,League of Legends Championship Series,North America,None,LCS 2022 Spring Playoffs,None,None,Primary,0,1,1,2022,1,1
9,LPL 2022 Summer,LPL/2022 Season/Summer Season,2022-06-10,2022-08-14,Tencent LoL Pro League,China,Offline/Online,LPL 2022 Summer,Summer,2,Primary,0,0,1,2022,1,1


In [7]:
teams = {}
for page in tournaments['OverviewPage']:
    scoreboard_games = get_scoreboard_games(f'T.OverviewPage="{page}"')
    scoreboard_games = scoreboard_games.sort_values(by='DateTime UTC').reset_index(drop=True)

    team_names = scoreboard_games[['Team1', 'Team2']].apply(pd.unique)
    team_names = list(set(list(team_names['Team1']) + list(team_names['Team2'])))
    league = page.split('/')[0]
    for name in team_names:
        if name not in teams:
            teams[name] = Team(name, league)

    proceed_rating(teams, scoreboard_games)

rating = get_rating(teams)
rating

,League,Win,Loss,WinRate,r,RD
Gen.G,LCK,73,23,0.760417,1283.698751,42.535600
T1,LCK,76,26,0.745098,1232.852709,46.173128
G2 Esports,LEC,42,20,0.677419,1210.338108,51.502710
Evil Geniuses.NA,LCS,57,29,0.662791,1205.942719,44.627048
Rogue (European Team),LEC,40,23,0.634921,1174.650062,51.538549
100 Thieves,LCS,42,28,0.600000,1171.412948,47.155461
Top Esports,LPL,78,44,0.639344,1170.792576,34.662074
Team Liquid,LCS,52,29,0.641975,1169.181441,46.788766
Victory Five,LPL,65,30,0.684211,1159.821938,41.237777
JD Gaming,LPL,71,36,0.663551,1159.677366,37.458230


In [8]:
tournaments = get_tournaments(f'L.League_Short="WCS" and T.Year=2022')
tournaments = tournaments.sort_values(by='DateStart').reset_index(drop=True)
tournaments

,Name,OverviewPage,DateStart,Date,League,Region,EventType,StandardName,Split,SplitNumber,TournamentLevel,IsQualifier,IsPlayoffs,IsOfficial,Year,DateStart__precision,Date__precision
0,LCK 2022 Regional Finals,LCK/2022 Season/Regional Finals,2022-09-01,2022-09-03,World Championship,Korea,None,LCK 2022 Regional Finals,None,None,Primary,1,0,1,2022,1,1
1,LPL 2022 Regional Finals Showmatch,LPL/2022 Season/Regional Finals/Showmatch,2022-09-01,2022-09-01,World Championship,China,Offline,LPL 2022 Regional Finals Showmatch,None,None,Showmatch,0,0,0,2022,1,1
2,LPL 2022 Regional Finals,LPL/2022 Season/Regional Finals,2022-09-02,2022-09-04,World Championship,China,Offline,LPL 2022 Regional Finals,None,None,Primary,1,0,1,2022,1,1
3,Worlds 2022 Play-In,2022 Season World Championship/Play-In,2022-09-29,2022-10-04,World Championship,International,Offline,Worlds 2022 Play-In,None,None,Primary,1,0,1,2022,1,1
4,Worlds 2022 Main Event,2022 Season World Championship/Main Event,2022-10-07,2022-11-05,World Championship,International,Offline,Worlds 2022 Main Event,None,None,Primary,0,0,1,2022,1,1


In [9]:
for page in tournaments['OverviewPage']:
    scoreboard_games = get_scoreboard_games(f'T.OverviewPage="{page}"')
    if scoreboard_games is None:
        continue
    scoreboard_games = scoreboard_games.sort_values(by='DateTime UTC').reset_index(drop=True)
    team_names = scoreboard_games[['Team1', 'Team2']].apply(pd.unique, axis=0, result_type='reduce')
    
    team_names = list(set(list(team_names['Team1']) + list(team_names['Team2'])))
    league = page.split('/')[0]
    for name in team_names:
        if name not in teams:
            teams[name] = Team(name, league)

    print(scoreboard_games.head())

    proceed_rating(teams, scoreboard_games)

rating = get_rating(teams)
rating

                      OverviewPage         Team1         Team2       WinTeam  \
0  LCK/2022 Season/Regional Finals       DWG KIA  Liiv SANDBOX       DWG KIA   
1  LCK/2022 Season/Regional Finals  Liiv SANDBOX       DWG KIA  Liiv SANDBOX   
2  LCK/2022 Season/Regional Finals       DWG KIA  Liiv SANDBOX       DWG KIA   
3  LCK/2022 Season/Regional Finals  Liiv SANDBOX       DWG KIA       DWG KIA   
4  LCK/2022 Season/Regional Finals    KT Rolster           DRX           DRX   

       LossTeam        DateTime UTC  Team1Score  Team2Score  Winner  \
0  Liiv SANDBOX 2022-09-01 08:08:00           1           0       1   
1       DWG KIA 2022-09-01 09:04:00           1           1       1   
2  Liiv SANDBOX 2022-09-01 10:01:00           2           1       1   
3  Liiv SANDBOX 2022-09-01 10:52:00           1           3       2   
4    KT Rolster 2022-09-02 08:05:00           0           1       2   

  Gamelength  Gamelength Number                                 Team1Bans  \
0      32:23   

,League,Win,Loss,WinRate,r,RD
Gen.G,LCK,73,23,0.760417,1283.698751,42.535600
T1,LCK,76,26,0.745098,1232.852709,46.173128
G2 Esports,LEC,42,20,0.677419,1210.338108,51.502710
Evil Geniuses.NA,LCS,62,31,0.666667,1207.022399,43.160041
Rogue (European Team),LEC,40,23,0.634921,1174.650062,51.538549
100 Thieves,LCS,42,28,0.600000,1171.412948,47.155461
Top Esports,LPL,78,44,0.639344,1170.792576,34.662074
Team Liquid,LCS,52,29,0.641975,1169.181441,46.788766
JD Gaming,LPL,71,36,0.663551,1159.677366,37.458230
Cloud9,LCS,44,28,0.611111,1157.814337,46.909859


In [10]:
team_names = [
    'Gen.G', 'T1', 'DWG KIA', 'DRX', 
    'JD Gaming', 'Top Esports', 'EDward Gaming', 'Royal Never Give Up',
    'G2 Esports', 'Rogue (European Team)', 'Fnatic', 'MAD Lions',
    'Cloud9', '100 Thieves', 'Evil Geniuses.NA',
    'DetonatioN FocusMe', 'LOUD', 'Beyond Gaming', 'Saigon Buffalo', 'Isurus', 'Istanbul Wildcats', 'Chiefs Esports Club'
]

rating.loc[team_names]

,League,Win,Loss,WinRate,r,RD
Gen.G,LCK,73,23,0.760417,1283.698751,42.535600
T1,LCK,76,26,0.745098,1232.852709,46.173128
DWG KIA,LCK,64,41,0.609524,1140.031656,38.789915
DRX,LCK,58,52,0.527273,1056.117850,36.851193
JD Gaming,LPL,71,36,0.663551,1159.677366,37.458230
Top Esports,LPL,78,44,0.639344,1170.792576,34.662074
EDward Gaming,LPL,66,42,0.611111,1105.455522,36.887084
Royal Never Give Up,LPL,73,42,0.634783,1133.253700,37.071423
G2 Esports,LEC,42,20,0.677419,1210.338108,51.502710
Rogue (European Team),LEC,40,23,0.634921,1174.650062,51.538549
